## Connect to IB

In [4]:
from ib_insync import *
util.startLoop()  

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

## Get tick by tick data for HSI trading  (RTH & ATH)

In [6]:
import pandas as pd
import time
from datetime import timedelta, datetime

# Constants
list_of_dates = ['20201029', '20201127', '20201230', '20210128', '20210225']
for END_DATE in list_of_dates:

    # Initialization
    contract = Future('HSI', END_DATE, 'HKFE', includeExpired=True)
    start_date = (datetime.strptime(END_DATE, '%Y%m%d') - timedelta(days=35)).strftime('%Y%m%d') 
    df = pd.DataFrame()
    temp = pd.DataFrame({'time': [0]})
    timing = f'{start_date} 04:00:00'

    while timing != f'{END_DATE} 16:00:00':
        bars = ib.reqHistoricalTicks(contract, startDateTime= timing, endDateTime='', numberOfTicks=1000, whatToShow='TRADES', useRth=False)
        if bars:
            timing = (bars[-1].time.replace(tzinfo=None) + timedelta(hours=8, seconds=1)).strftime('%Y%m%d %H:%M:%S')
            temp = util.df(bars)[['time','price','size']]

            df = pd.concat([df, temp])

            print(timing)

        else:
            timing = (datetime.strptime(timing, '%Y%m%d %H:%M:%S') + timedelta(seconds=1)).strftime('%Y%m%d %H:%M:%S')
            
    df.to_csv(f'{END_DATE}newhsitick.csv')

20200924 09:42:02
20200924 10:08:50
20200924 10:32:44
20200924 11:09:33
20200924 11:54:17
20200924 13:54:06
20200924 14:49:44
20200924 15:36:08
20200924 16:09:28
20200924 16:29:59
20200924 23:12:05
20200925 09:20:04
20200925 09:33:54
20200925 09:45:27
20200925 10:00:19
20200925 10:18:08
20200925 10:38:40
20200925 10:58:21
20200925 11:20:46
20200925 11:44:12
20200925 13:04:22
20200925 13:30:05
20200925 14:00:54
20200925 14:20:47


Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm.nj; hfarm; cashfarm; usfarm; hkhmds; ushmds; secdefnj.


KeyboardInterrupt: 

In [3]:
import pandas as pd
import time
from datetime import timedelta, datetime

END_DATE = '20210225'
(datetime.strptime(END_DATE, '%Y%m%d') - timedelta(days=35)).strftime('%Y%m%d')

'20210121'

## Save as csv

In [51]:
df.to_csv('202102hsitick.csv', index=False)

## Get 6 months HSI 5sec bar for first 1 hr

In [ ]:
def getdatesfuture(date):
    # get list of dates 
    from datetime import timedelta, datetime
    todate = datetime.strptime(date, '%Y%m%d')
    delta = timedelta(days=35)    
    list_of_dates = []

    for i in range(delta.days + 1):
        day = todate - delta + timedelta(days=i)
        list_of_dates.append(day.strftime("%Y%m%d"))

    return list_of_dates


def getdatafuture(date):
    # get traded price of future
    import pandas as pd

    contract = Future('HSI', date, 'HKFE', includeExpired=True)

    df = pd.DataFrame()

    list_of_dates = getdatesfuture(date)

    for i in list_of_dates:
        bars = ib.reqHistoricalData(contract, endDateTime=f'{i} 09:45:00', durationStr='1800 S', barSizeSetting='1 secs', whatToShow='BID', useRTH=True)
        dfbid = util.df(bars)

        bars = ib.reqHistoricalData(contract, endDateTime=f'{i} 09:45:00', durationStr='1800 S', barSizeSetting='1 secs', whatToShow='ASK', useRTH=True)
        dfask = util.df(bars)

        merged = pd.merge(dfbid,dfask, left_index=True, right_index=True,suffixes=('bid','ask'))
        temp = merged[['datebid', 'openbid', 'highbid', 'lowbid', 'openask', 'highask', 'lowask']]

        bars = ib.reqHistoricalData(contract, endDateTime=f'{i} 09:45:00', durationStr='1800 S', barSizeSetting='1 secs', whatToShow='TRADES', useRTH=True)
        dftrades = util.df(bars)

        merged = pd.merge(temp,dftrades, left_index=True, right_index=True,suffixes=('','trades'))
        temp = merged[['datebid', 'openbid', 'highbid', 'lowbid', 'openask', 'highask', 'lowask', 'open', 'high', 'low', 'close', 'volume', 'average', 'barCount']]

        df = pd.concat([df, temp])
        print(i)
        
        time.sleep(30)  
        
    return(df)

#Get recent 2 years data
import pandas as pd
import time
df = pd.DataFrame()

for year in range(2020,2021):
    for month in range(6,13):
        for day in range(27,31):
            try:
                df = df.append(getdatafuture(f'{year}{month+1:02d}{day}'))
            except:
                pass

## Get HSI 30secs bar inside and outside RTH

In [ ]:
def getdatesfuture(date):
    # get list of dates 
    from datetime import timedelta, datetime
    todate = datetime.strptime(date, '%Y%m%d')
    delta = timedelta(days=35)    
    list_of_dates = []

    for i in range(delta.days + 1):
        day = todate - delta + timedelta(days=i)
        list_of_dates.append(day.strftime("%Y%m%d"))

    return list_of_dates


def getdatafuture(date):
    # get traded price of future
    import pandas as pd

    contract = Future('HSI', date, 'HKFE', includeExpired=True)

    df = pd.DataFrame()

    list_of_dates = getdatesfuture(date)

    for i in list_of_dates:

        bars = ib.reqHistoricalData(contract, endDateTime=f'{i} 16:30:00', durationStr='1 D', barSizeSetting='30 secs', whatToShow='TRADES', useRTH=False)
        temp = util.df(bars)
        temp = temp[['date', 'open', 'high', 'low', 'close', 'volume', 'average', 'barCount']]
        
        try:
            check = df.iloc[-1].equals(temp.iloc[-1])
        except:
            check = False

        if not check:
            df = pd.concat([df, temp])
            print(i)
                
    return(df)

#Get recent 2 years data
import pandas as pd
import time
df = pd.DataFrame()

for year in range(2019,2021):
    for month in range(1,13):
        for day in range(25,31):
            try:
                df = df.append(getdatafuture(f'{year}{month+1:02d}{day}'))
            except:
                pass